# Data Wrangling & EDA

##### Kimberly Liu & Isaac Tabor

### March Madness Data

**Part 1: What is our data?**

We believe the information and variables highlighted from the following datasets will help us build a simple prediction model:

- *MTeams.csv* and *WTeams.csv* contain **Team ID** and **Team Names**

- *MNCAATourneySeeds.csv* and *WNCAATourneySeeds.csv* contain **tournament seeds since 1984-85 season**. Key to note: We will not know which 68 teams will be in the tournament, or what seeds  are until Selection Sunday on March 16, 2025.

- *MRegularSeasonCompactResults.csv* and *WRegularSeasonCompactResults.csv* contain **Final scores of all regular season, conference tournament, and NCAA® tournament games since 1984-85 season**

- *MSeasons.csv* and *WSeasons.csv* contain **Season-level details including dates and region names**

In the end, we plan to generate our predictions from a machine learning model in format outlined in *SampleSubmissionStage1.csv*



**Part 2: How will these data be useful for studying the phenomenon we're interested in?**

We have collected a large amount of data of historical NCAA basketball games and teams going back many years. We intend to use it to build a machine learning model to predict March Madness outcomes.

We have data on both men's and women's data currently, with files starting with M containing only data pertaining to men's data, and files starting with W containing only women's data (e.g. MCities, WConferences). MTeamSpellings and WTeamSpellings will help us map TeamID to the team.

All of the files are currently complete through January 28th of the current season. This data was compiled into a Kaggle dataset for a March Madness ML competition largely from Kenneth Massey and Jeff Sonas of Sonas Consulting.


**Part 3: What are the challenges we've resolved or expect to face in using them?**

First clone the GitHub repo:

In [3]:
! git clone https://github.com/kimberlyyliuu/DS3001-Project/

Cloning into 'DS3001-Project'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 93 (delta 20), reused 11 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 12.63 MiB | 3.17 MiB/s, done.
Resolving deltas: 100% (20/20), done.


Next, load and merge basic data. You may have to adjust to your unique file path:

In [7]:
import pandas as pd

MTeams = pd.read_csv("/content/DS3001-Project/data/MTeams.csv")
MNCAATournamentSeeds = pd.read_csv("/content/DS3001-Project/data/MNCAATourneySeeds.csv")
MRegularSeasonCompactResults = pd.read_csv("/content/DS3001-Project/data/MRegularSeasonCompactResults.csv")
MSeasons = pd.read_csv("/content/DS3001-Project/data/MSeasons.csv")

In [8]:
MTeams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025


In [9]:
MNCAATournamentSeeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [10]:
MRegularSeasonCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [11]:
MSeasons.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/02/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [12]:
# Assuming you have loaded the DataFrames as MTeams, MNCAATournamentSeeds,
# MRegularSeasonCompactResults, and MSeasons

# 1. Merge MTeams for winning teams
merged_df = pd.merge(MRegularSeasonCompactResults, MTeams, left_on='WTeamID', right_on='TeamID', suffixes=('', '_winner'))

# 2. Merge MTeams for losing teams
merged_df = pd.merge(merged_df, MTeams, left_on='LTeamID', right_on='TeamID', suffixes=('', '_loser'))

# 3. Merge MNCAATournamentSeeds
merged_df = pd.merge(merged_df, MNCAATournamentSeeds, on=['Season', 'TeamID'], how='left')  # Left merge to keep all regular season games

# 4. Merge MSeasons
merged_df = pd.merge(merged_df, MSeasons, on='Season')

# Now 'merged_df' contains all the combined information
merged_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,TeamID,TeamName,...,TeamID_loser,TeamName_loser,FirstD1Season_loser,LastD1Season_loser,Seed,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,20,1228,81,1328,64,N,0,1228,Illinois,...,1328,Oklahoma,1985,2025,W03,10/29/1984,East,West,Midwest,Southeast
1,1985,25,1106,77,1354,70,H,0,1106,Alabama St,...,1354,S Carolina St,1985,2025,NaN,10/29/1984,East,West,Midwest,Southeast
2,1985,25,1112,63,1223,56,H,0,1112,Arizona,...,1223,Houston Chr,1985,2025,X10,10/29/1984,East,West,Midwest,Southeast
3,1985,25,1165,70,1432,54,H,0,1165,Cornell,...,1432,Utica,1985,1987,NaN,10/29/1984,East,West,Midwest,Southeast
4,1985,25,1192,86,1447,74,H,0,1192,F Dickinson,...,1447,Wagner,1985,2025,Z16,10/29/1984,East,West,Midwest,Southeast


Now, we have a basic merged dataset to start with. Kimberly, if you have a better one, we can use that or merge those too.